In [2]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.utils import util

In [3]:
pickled_lens_list = os.path.join(config.machine.dir_01, '01_hlwas_sim_detectable_lens_list')
lens_list = util.unpickle(pickled_lens_list)

# Detectable and candidate lens counts

Number of detectable strong lenses per square degree of simulated area, candidate strong lenses

In [ ]:
# combine CSVs from each run into one CSV
data_dir = config.machine.dir_00
total_csv = os.path.join(data_dir, 'total_pop.csv')
detectable_csv = os.path.join(data_dir, 'detectable_pop.csv')
util.delete_if_exists(total_csv)
util.delete_if_exists(detectable_csv)
total_df = util.combine_all_csvs(data_dir, 'total', total_csv)
detectable_df = util.combine_all_csvs(data_dir, 'detectable', detectable_csv)

In [ ]:
runs = len(glob(os.path.join(data_dir, "total_pop_*.csv")))
total_area = 5 * runs
print(f'{runs} runs of 5 sq. deg. each gives {total_area} sq. deg. total')

In [ ]:
print(f'{len(total_df)} candidate strong lenses')
print(f'{len(detectable_df)} detectable strong lenses')

fraction_detectable = len(detectable_df) / len(total_df)
print(f'Fraction of candidate strong lenses that are detectable: {fraction_detectable:.5f}')

In [ ]:
total_per_sq_deg = len(total_df) / total_area
print(f'Candidate strong lenses per sq. deg.: {round(total_per_sq_deg)}')
det_per_sq_deg = len(detectable_df) / total_area
print(f'Detectable strong lenses per sq. deg.: {round(det_per_sq_deg)}')

In [ ]:
det_per_exposure = det_per_sq_deg * 0.281
print(f'Detectable strong lenses per exposure: {round(det_per_exposure)}')

# How much work are the various filters doing?

`filter_1` is Einstein radius and half-light radius, and `filter_2` is SNR > 10. The other detectability criteria are combined into the difference between candidate and detectable.

In [ ]:
filters = util.unpickle_all(config.machine.dir_00, 'filtered_sample_')

# Subhalo statistics

In [ ]:
stats_list = util.unpickle_all(os.path.join(config.machine.dir_02, 'stats'), 'subhalo_stats_')

In [ ]:
original_einstein_radii = [d['original_einstein_radius'] for d in stats_list]
adjusted_einstein_radii = [d['adjusted_einstein_radius'] for d in stats_list]
percent_change_einstein_radii = [d['percent_change_einstein_radius'] for d in stats_list]
effective_lensing_masses = [d['effective_lensing_mass'] for d in stats_list]
adjusted_lensing_masses = [d['adjusted_lensing_mass'] for d in stats_list]
percent_change_lensing_masses = [d['percent_change_lensing_mass'] for d in stats_list]
total_masses_subhalos_within_einstein_radius = [d['total_mass_subhalos_within_einstein_radius'] for d in stats_list]
total_subhalo_masses = [d['total_subhalo_mass'] for d in stats_list]
percent_subhalo_masses_within_einstein_radius = [d['percent_subhalo_mass_within_einstein_radius'] for d in stats_list]

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].hist(percent_change_einstein_radii, bins=50, color='k', alpha=0.5)
ax[0].set_xlabel('Percent Change in Einstein Radius')
ax[0].set_ylabel('Number of Lenses')
ax[1].hist(percent_change_lensing_masses, bins=50, color='k', alpha=0.5)
ax[1].set_xlabel('Percent Change in Lensing Mass')
ax[1].set_ylabel('Number of Lenses')
ax[2].hist(percent_subhalo_masses_within_einstein_radius, bins=50, color='k', alpha=0.5)
ax[2].set_xlabel('Percent of Subhalo Mass within Einstein Radius')
ax[2].set_ylabel('Number of Lenses')
plt.tight_layout()
plt.show()